# GNN notebook

In [22]:
from typing import List
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import jraph
import optax
import data_utils
import os
import pathlib
import h5py
from torch.utils.data import Dataset, DataLoader, default_collate


os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [23]:
# train_dir = "/global/cfs/projectdirs/m3246/mingfong/transfer-learning/delphes_train_set/"   # original fullsim
# train_dir = "/pscratch/sd/m/mingfong/transfer-learning/delphes_train_processed/"            # pretrain processed
train_dir = "/pscratch/sd/m/mingfong/transfer-learning/fullsim_train_processed/"              # fullsim processed
train_path = pathlib.Path(train_dir)
# train_file_names = os.listdir(train_dir)
train_filepaths = [p for p in train_path.glob("*.h5")]
train_filepaths

[PosixPath('/pscratch/sd/m/mingfong/transfer-learning/fullsim_train_processed/preprocessed_train.h5')]

In [24]:
# create GraphsTuple
MAX_CONSTITUENTS = 200      # nodes per graph
FEATURE_NAME_TO_INDEX = {   # (n, constituent, feature) <- feature index name mapping for data
  "eta_flip": 0,
  "phi_rot": 1,
  "log_pt": 2,
  "log_energy": 3,
  "lognorm_pt": 4,
  "lognorm_energy": 5,
  "radius": 6,
}

with h5py.File(train_filepaths[0], "r") as f:
  data = f["data"]
  labels = f["labels"]
  print(f.keys())
  feature_names = list(f["data"].keys())
  file_len = len(f["labels"])
  print(f"file_len = {file_len}")
  for feature_name in feature_names:
    print(feature_name, data[feature_name].shape)
  
  node_features, edge_features, senders, receivers, globals, targets = None, None, None, None, None, None
  # for i in range(file_len):
    # node_features = 
  

<KeysViewHDF5 ['data', 'labels']>


AttributeError: 'Dataset' object has no attribute 'keys'

In [5]:
# classification example
# https://github.com/google-deepmind/jraph/blob/51f5990104f7374492f8f3ea1cbc47feb411c69c/jraph/ogb_examples/train.py#L89

In [3]:
class H5GraphDataset(Dataset):
  """
  Turns h5 files into GraphsTuple objects
  """
  def __init__(self, filepaths: List[str], max_rows:int = None):
    self.filepaths = filepaths